In [1]:
from PIL import Image
import numpy as np
import os
import cv2

In [2]:
import glob

In [3]:
all_files = sorted(glob.glob('un_resized_images/fidels/*/*.jpg', recursive=True))
#all_files_2 = glob.glob('un_resized_2/fidels/*/*.jpg', recursive=True)

In [4]:
Image.open(all_files[0]).size

j = []
k = []
for i in all_files:
    #print("the size of " , all_files.index(i), "\t" , Image.open(i).size) 
    if(Image.open(i).size == Image.open(all_files[0]).size):
        j.append(all_files.index(i))
    else:
        k.append(all_files.index(i))
        

#print(j)
print(k)
Image.open(all_files[225]).size

[225]


(720, 446)

In [5]:
all_x = []
all_y = []


In [6]:
for i in all_files:
    if(i == all_files[225]):
        all_x.append(np.array(Image.open(i).convert('L')).reshape(-1, 1)[:315360, :])
    else:
        all_x.append(np.array(Image.open(i).convert('L')).reshape(-1, 1))
    all_y.append(i.split('/')[2])


In [7]:
X = np.array(all_x)

In [8]:
y = np.array(all_y).reshape(-1, 1)

In [9]:
X.shape

(411, 315360, 1)

In [10]:
X = X.squeeze()
X.shape # shape of X = (m,n )

(411, 315360)

In [11]:
y.shape # shape of lable y = (m, 1)

(411, 1)

In [12]:
X.shape

(411, 315360)

In [13]:
y.shape
y[0]
for i in y:
    print(i[0])

00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
00
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
01
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
02
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
03
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
04
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
05
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
06
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
07
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
08
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
09
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
1

In [14]:
y_to_b = []
for i in y:
    row = [0]*14
    row[int(i[0])-1] = 1
    y_to_b.append(row)
y_to_b = np.array(y_to_b)

In [15]:
print(y_to_b.shape)
print(X.shape)

(411, 14)
(411, 315360)


In [16]:
class model():
    def __init__(self, X, num_class): 
        self.weights = np.random.rand(X.shape[1], num_class) 
        #self.weights = np.zeros((X.shape[1], num_class)) 
        self.biases = np.zeros(shape = (1, num_class))
        self.X = X
        self.num_class = num_class
         
    def forward(self, X, w, b):
        return np.matmul(X, w) + b
    
    def softmax(self, z):
        z -= np.max(z, axis = 1).reshape(-1, 1)
        return np.exp(z) / np.array(np.sum(np.exp(z), axis = 1)).reshape(-1, 1)
        
    def cost(self, softmax, y):
        #m = y.shape[0]
        #cost = (- 1 / m) * np.sum(y * np.log(softmax) + (1 - y) * (np.log(1 - softmax)))  # compute cost
        
        #loss = (y * np.log(softmax)) + (1-y) * (np.log(1 - softmax))
        loss = np.multiply(y, np.log(softmax)) + np.multiply(y, np.log(1 - softmax))
        num_image = y.shape[0]
        cost = - np.sum(loss) / num_image
        cost = np.squeeze(cost)
        return cost
    
    def gradient_cost(self, X, softmax, y):
        loss = self.loss
    #this loss1 is the cross entropy cost
    
    def cross_entropy_cost(self, softmax, y):
        cost = -np.sum(np.multiply(y, np.log(softmax + 0.01)))
        #cost = -np.sum(np.multiply(y, np.log(softmax[:, y])))
        return cost
    
         
    def backward(self, X, softmax, y):
        m = X.shape[1]
        dw = (1 / m) * np.dot(X.T, (softmax - y))
        db = (1 / m) * np.sum(softmax - y)
        return dw, db
    
    def update_parameters(self, dw, db, alpha, w_old, b_old):
        w_new = w_old - alpha * dw
        b_new = b_old - alpha * db 
        return w_new, b_new
    
    def update_parameters_2(self, X, cost, lr, w_old, b_old):
        
        w_new = w_old - np.dot(lr, np.multiply(X, cost))
        b_new = b_old - np.dot(lr, np.sum(cost))
        
        return w_new, b_new


In [129]:
num_class = 14
model = model(X, num_class)
w = model.weights
b = model.biases


In [130]:
print(w.shape)
print(w)

(315360, 14)
[[0.09566345 0.41202512 0.98278465 ... 0.46277345 0.34680876 0.07586337]
 [0.13158312 0.98275836 0.20716539 ... 0.63627812 0.78645695 0.85093602]
 [0.10182015 0.5378875  0.44339566 ... 0.97062238 0.47733524 0.35978219]
 ...
 [0.95675694 0.2201281  0.46929072 ... 0.25646801 0.79764721 0.12060023]
 [0.739658   0.25367218 0.41362845 ... 0.17634689 0.33815526 0.7305169 ]
 [0.89049757 0.37436082 0.94773182 ... 0.57973606 0.63734868 0.21051034]]


In [131]:
print(b.shape)
print(b)

(1, 14)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# The Forward Pass

In [132]:

a = model.forward(X, w, b)


In [133]:
print("The shape of the output array becomes", a.shape)
print("\nThe forward output of all the image\n")
print(a)
#print("\nThe forward output of the 20th image how much it is means the prediction through the 14 class")
#print(a[20])

The shape of the output array becomes (411, 14)

The forward output of all the image

[[389491.86568379 395709.93407414 391388.81714302 ... 385761.18176725
  392395.88639736 393672.58594741]
 [407668.07161205 412349.57568827 408277.5258752  ... 419032.06171371
  407324.52681541 420944.90325016]
 [429656.06226668 431434.29654573 428408.02517499 ... 431971.8516439
  431474.61281957 433762.99496916]
 ...
 [570063.28737604 577485.62852097 586202.47865217 ... 579950.71570941
  570101.63527987 574080.84886496]
 [598548.2917533  609915.0946554  610397.19815486 ... 608256.87634827
  604807.74059102 606285.90604652]
 [581947.71364235 582626.22825886 565899.08914219 ... 574384.64575462
  580301.67932183 581747.25777775]]


Expected Result

The shape of the output array becomes (411, 14)

The forward output of all the image

The forward output of the 20th image how much it is means the prediction through the 14 class
[474244.45342753 480183.38392286 471674.62918    478515.06985859
 464605.62585467 477473.93646667 473011.59488537 478671.92048778
 461979.22081921 470068.8819789  480149.49769161 469849.74504296
 466378.11909758 472072.12665338]


# The Softmax

In [134]:
softmax = model.softmax(a)
print("The shape of the prediction of an array output becomes", softmax.shape)
print("\n", softmax)

print("\nchecking the softmax by summing their values")
print(np.sum(softmax[165]))

The shape of the prediction of an array output becomes (411, 14)

 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

checking the softmax by summing their values
1.0


In [135]:


print("\ndisplay all the prediction, changing the forward output of all the 14 class to the probabilit 1 or 0")
#print(prediction)

'''
#the class of the image you want to check whether your model predicts okey or not by checking with your file
im = int(input("\nEnter here index of your image "))

print("\nThe prediction of the ", im , "th image, which class it belongs by the probabilty of 1 or 0")
print(prediction[im])

pred_tolist = prediction[im].tolist()
clas = -1
for i in pred_tolist:
    if i == 1:
        clas = pred_tolist.index(i)

print("\nThe ", im, "th image belongs to the", clas+1, "Class")
Image.open(all_files[im])
'''


display all the prediction, changing the forward output of all the 14 class to the probabilit 1 or 0


'\n#the class of the image you want to check whether your model predicts okey or not by checking with your file\nim = int(input("\nEnter here index of your image "))\n\nprint("\nThe prediction of the ", im , "th image, which class it belongs by the probabilty of 1 or 0")\nprint(prediction[im])\n\npred_tolist = prediction[im].tolist()\nclas = -1\nfor i in pred_tolist:\n    if i == 1:\n        clas = pred_tolist.index(i)\n\nprint("\nThe ", im, "th image belongs to the", clas+1, "Class")\nImage.open(all_files[im])\n'

# The Loss

In [137]:
#loss = model.cross_entropy_cost(softmax, y_to_b)
loss = model.cost(softmax, y_to_b)

print("the loss is", loss)


the loss is nan


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in multiply


# Back Propagation

In [309]:
dw, db = model.backward(X, softmax, y_to_b)

In [310]:
print(dw)
print(dw.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(315360, 14)


In [311]:
print(db)
print(db.shape)

[-3.48807712e-05  7.61039320e-05 -5.39066464e-05 -1.58548960e-05
 -2.53681517e-05 -2.21968544e-05  1.58547768e-05  2.85390362e-05
  1.90258813e-05 -1.90271138e-05  1.10985287e-04 -1.58548960e-05
 -8.87874176e-05  2.53678336e-05]
(14,)


# Update Prameters

In [329]:
u_w1, u_b1 = model.update_parameters(dw, db, 0.01)
w, b = model.update_parameters_2(X, cost, 0.01, w, b)

ValueError: operands could not be broadcast together with shapes (315360,14) (315360,411) 

In [313]:
print(u_w1)
print(u_w1.shape)

[[0.78607806 0.27039203 0.713321   ... 0.47925283 0.78858469 0.74270402]
 [0.20535854 0.74668004 0.31798676 ... 0.7847037  0.92165342 0.91258049]
 [0.73720568 0.36417796 0.58936748 ... 0.13458229 0.68764747 0.46957446]
 ...
 [0.07003252 0.61303119 0.32644886 ... 0.86479532 0.88016751 0.75066871]
 [0.10480946 0.36189196 0.70345137 ... 0.96389154 0.82944581 0.43711838]
 [0.29030474 0.91193542 0.35683578 ... 0.57791056 0.59492973 0.8302037 ]]
(315360, 14)


In [314]:
print(u_b1)
print(u_b1.shape)

[[ 3.48807712e-07 -7.61039320e-07  5.39066464e-07  1.58548960e-07
   2.53681517e-07  2.21968544e-07 -1.58547768e-07 -2.85390362e-07
  -1.90258813e-07  1.90271138e-07 -1.10985287e-06  1.58548960e-07
   8.87874176e-07 -2.53678336e-07]]
(1, 14)


# The Second forward pass

In [315]:
forward2 = model.forward(X, u_w1, u_b1)

In [319]:
softmax22 = model.softmax(forward2)

In [320]:
loss22 = model.cross_entropy_cost(softmax22, y_to_b)

In [321]:
print(loss22)

1772.7318130027202


# For Loops

In [113]:
num_class = 14
model = model(X, num_class)


In [114]:
w = model.weights
b = model.biases

In [116]:

#print("initial weight\n\t", w)
#print("\ninitial bias\n\t", b )
for i in range(10):
    #print("\nweight ", i , "\n\t", w)
    #print("\nbias ", i, "\n\t", b )
    forward = model.forward(X, w, b)
    soft = model.softmax(forward)
    #cost = model.cross_entropy_cost(soft, y_to_b)
    cost = model.cost(soft, y_to_b)
    print("The Loss in Step ", i , " is \t", cost)
    dw, db = model.backward(X, soft, y_to_b)
    w, b = model.update_parameters(dw, db, 0.001, w, b)
    #w, b = model.update_parameters_2(X, cost, 0.01, w, b)
    
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in multiply


The Loss in Step  0  is 	 nan
The Loss in Step  1  is 	 nan
The Loss in Step  2  is 	 nan
The Loss in Step  3  is 	 nan
The Loss in Step  4  is 	 nan
The Loss in Step  5  is 	 nan
The Loss in Step  6  is 	 nan
The Loss in Step  7  is 	 nan
The Loss in Step  8  is 	 nan
The Loss in Step  9  is 	 nan


In [92]:
import tensorflow as tf
import numpy as np

In [93]:
XX = tf.placeholder(tf.float32, shape = (411, 720*438*1))

In [94]:
yy = tf.placeholder(tf.float32, shape=[411, 14])

In [95]:
ww = tf.Variable(tf.zeros((720*438*1, 14)))

In [96]:
bb = tf.Variable(tf.zeros((1, 14)))

In [97]:
print(X.shape)
print(XX.shape)

(411, 315360)
(411, 315360)


In [98]:
print(ww.shape)

print(bb.shape)

(315360, 14)
(1, 14)


In [99]:
print(yy.shape)

(411, 14)


In [100]:
mat = tf.matmul(XX, ww)
logit = tf.add(mat, bb)

In [101]:
logit.shape

TensorShape([Dimension(411), Dimension(14)])

In [106]:
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=yy, logits=logit))

In [103]:
train = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [104]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(15):
        losss,_ = sess.run([loss,train], feed_dict={XX:X, yy:y_to_b})
        print(str(losss))

1084.6522
255266.97
1000963.9
3412283.2
343078.38
3390.4277
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [112]:
def tf_loss(logits, labels):
    cross_entropy_per_number = tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels, logits = logits)
    cross_entropy = tf.reduce_mean(cross_entropy_per_number)
    tf.add_to_collection("loss", cross_entropy)
    return cross_entropy


In [114]:
print(tf_loss(logit, 

Tensor("Mean_1:0", shape=(), dtype=float32)


In [115]:
def evaluation(logits, labels):
    prediction = tf.argmax(logits, 2)
    actual = tf.argmax(labels, 2)
    equal = tf.equal(prediction, actual)
    # equal = tf.reduce_all(equal, 1)
    accuracy = tf.reduce_mean(tf.cast(equal, tf.float32), name="accuracy")
    return accuracy


In [118]:
def train(loss, learning_rate=0.00001):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    print(train_op)
    return train_op

In [119]:
train(tf_loss(logit, yy))

name: "GradientDescent_14"
op: "NoOp"
input: "^GradientDescent_14/update_Variable_4/ApplyGradientDescent"
input: "^GradientDescent_14/update_Variable_5/ApplyGradientDescent"



<tf.Operation 'GradientDescent_14' type=NoOp>